In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
from ydata_profiling import ProfileReport

c:\Users\CARLES\anaconda3\envs\financialkeepcoders\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
key_path = key_path
project = project_id

# Sources
currency_table = 'silver_currency_data'
currency_dataset = 'silver'
indicators_table = 'silver_indicators'
indicators_dataset = 'silver'
ticker_info_table = 'silver_ticker_info'
ticker_info_dataset = 'silver'
macro_data_table = 'silver_fred_macro_data'
macro_data_dataset = 'silver'

# Tables id
table_conca_currency = f'{project}.{currency_dataset}.{currency_table}'
table_conca_indicators = f'{project}.{indicators_dataset}.{indicators_table}'
table_conca_ticker_info = f'{project}.{ticker_info_dataset}.{ticker_info_table}'
table_conca_macro_data = f'{project}.{macro_data_dataset}.{macro_data_table}'

# Loading tables
table_to_save = 'gold_main_sp500'
dataset_to_save = 'gold'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [12]:
df = bigquery.run_query(
    f"""
    SELECT
        indi.*,
        info.industry,
        info.sector,
        cu.USDEUR,
        cu.USDJPY,
        cu.USDGBP,
        cu.USDCHF,
        cu.USDCNY,
        mc.BOPGSTB,
        mc.CPIAUCSL,
        mc.FEDFUNDS,
        mc.GDP,
        mc.PPIACO,
        mc.RSAFS,
        mc.UMCSENT,
        mc.UNRATE
    FROM {table_conca_indicators} AS indi
    INNER JOIN {table_conca_ticker_info} AS info
        ON info.ticker = indi.ticker    
    INNER JOIN {table_conca_currency} AS cu
        ON indi.date = cu.Date
    INNER JOIN {table_conca_macro_data} AS mc
        ON indi.date = mc.Date
    """
)

df

,date,open,high,low,close,adjclose,volume,ticker,id,SMA_20,...,AUDUSD,USDCHF,BOPGSTB,CPIAUCSL,FEDFUNDS,GDP,PPIACO,RSAFS,UMCSENT,UNRATE
0,2018-07-25 00:00:00+00:00,101.199997,104.400002,97.900002,101.900002,101.900002,14420,SYRS,53d91877c84ae3cb39ffbe66bff30a93,44.862501,...,0.742622,0.99330,-49673.0,251.214,1.91,20798.730,204.3,502255.0,97.9,3.8
1,2018-07-27 00:00:00+00:00,36.250000,36.549999,36.000000,36.000000,32.457176,225500,HFWA,55214eba155401b9b4bd898af7c9750c,25.439000,...,0.738007,0.99402,-49673.0,251.214,1.91,20798.730,204.3,502255.0,97.9,3.8
2,2018-07-11 00:00:00+00:00,24.840000,25.020000,24.830000,24.950001,21.237066,203600,FCPT,4adc8bceb36e7f05e2f4987dba827c87,25.499501,...,0.741400,0.99253,-49673.0,251.214,1.91,20798.730,204.3,502255.0,97.9,3.8
3,2018-07-03 00:00:00+00:00,25.930000,26.500000,25.860001,25.870001,23.621937,9400,FBIZ,753593420c16af869d3bd4b3e0816601,25.379000,...,0.733891,0.99330,-49673.0,251.214,1.91,20798.730,204.3,502255.0,97.9,3.8
4,2018-07-31 00:00:00+00:00,18.750000,18.799999,18.450001,18.650000,18.650000,186798,MODN,6d332c9895b94e7cf88f137868197f6c,21.899000,...,0.740730,0.98808,-49673.0,251.214,1.91,20798.730,204.3,502255.0,97.9,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7030086,2019-04-24 00:00:00+00:00,110.989998,114.000000,110.160004,113.860001,111.351845,773600,PLCE,d428f46b4445abc166b4e2d7bb523bc9,65.915500,...,0.709668,1.02015,-49592.0,255.233,2.42,21384.775,202.1,509564.0,97.2,3.7
7030087,2019-04-30 00:00:00+00:00,33.799999,34.099998,32.299999,32.599998,32.599998,10250,CASI,282005388c7c8562a6a7de0d5dcb93a0,23.501999,...,0.705517,1.01930,-49592.0,255.233,2.42,21384.775,202.1,509564.0,97.2,3.7
7030088,2019-04-24 00:00:00+00:00,59.470001,62.500000,59.240002,61.599998,59.910145,3200,BFC,407e51c149a2c3871c138a193f30f13c,44.959000,...,0.709668,1.02015,-49592.0,255.233,2.42,21384.775,202.1,509564.0,97.2,3.7
7030089,2019-04-02 00:00:00+00:00,150.750000,150.750000,134.550003,143.774994,143.774994,1680,VERO,cc7935ad3cc0bf245b30939164251d80,29.083750,...,0.710990,0.99920,-49592.0,255.233,2.42,21384.775,202.1,509564.0,97.2,3.7


In [8]:
df_filtered = df[['sector', 'industry']]
df_filtered

,sector,industry
0,Technology,Software—Infrastructure
1,Consumer Cyclical,Restaurants
2,Technology,Software—Infrastructure
3,Technology,Electronic Components
4,Financial Services,Banks—Regional
...,...,...
7030086,Healthcare,Medical Care Facilities
7030087,Industrials,Rental & Leasing Services
7030088,Technology,Semiconductors
7030089,Real Estate,REIT - Mortgage


In [9]:
# Ejecutamos ProfileReport para realizar el EDA
profile = ProfileReport(df_filtered, title="sectors")

# Como es muy pesado, lo guardamos en un archivo HTML
profile.to_file(r'C:\Users\CARLES\Downloads\sectors.html')

c:\Users\CARLES\anaconda3\envs\financialkeepcoders\lib\site-packages\ydata_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 116.61it/s]


In [13]:
# Eliminamos la columna industry ya que hay muchos valores vacíos y muchos valores distintos
df = df.drop(columns='industry')

# Rellenamos los vacíos con others
df['sector'].fillna('others', inplace=True)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7030091 entries, 0 to 7030090
Data columns (total 47 columns):
 #   Column             Dtype              
---  ------             -----              
 0   date               datetime64[us, UTC]
 1   open               float64            
 2   high               float64            
 3   low                float64            
 4   close              float64            
 5   adjclose           float64            
 6   volume             Int64              
 7   ticker             object             
 8   id                 object             
 9   SMA_20             float64            
 10  EMA_50             float64            
 11  WILLR              float64            
 12  ATR                float64            
 13  ADX                float64            
 14  CCI                float64            
 15  OBV                float64            
 16  RSI                float64            
 17  CDL_DOJI           Int64              
 18  CD

In [16]:
nulos_por_columna = df.isnull().sum()
nulos_por_columna

date                    0
open                    0
high                    0
low                     0
close                   0
adjclose                0
volume                  0
ticker                  0
id                      0
SMA_20                  0
EMA_50                  0
WILLR                   0
ATR                     0
ADX                     0
CCI                     0
OBV                     0
RSI                     0
CDL_DOJI                0
CDL_HAMMER              0
CDL_MORNING_STAR        0
CDL_SHOOTING_STAR       0
CDL_ENGULFING           0
CDL_LONGLINE            0
bb_bbh                  0
bb_bbm                  0
bb_bbl                  0
daily_return            0
macd                    0
macd_signal             0
macd_hist               0
stoch                   0
stoch_d                 0
sector                  0
EURUSD                  0
GBPUSD                  0
USDCNY               3100
JPYUSD                  0
AUDUSD                  0
USDCHF      

In [20]:
# Convertimos las variables categoricas en dummies
columns_to_encode = ['sector']
categorical_df = pd.DataFrame([], columns=[])

# Aplicamos el one hot encoding
for col in columns_to_encode:
  new_cols_oneHot = pd.get_dummies(df[col], prefix=col)*1.0 # multiplico por 1.0 para convertir True/False a números directamente
  categorical_df = pd.concat([categorical_df, new_cols_oneHot], axis=1)

# Unimos en el mismo df
df_final = pd.concat([df, categorical_df], axis=1, join='inner')

# Eliminamos las variables categoricas que ya hemos transformado
df_final = df_final.drop(columns=columns_to_encode)

df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7030091 entries, 0 to 7030090
Data columns (total 58 columns):
 #   Column                         Dtype              
---  ------                         -----              
 0   date                           datetime64[us, UTC]
 1   open                           float64            
 2   high                           float64            
 3   low                            float64            
 4   close                          float64            
 5   adjclose                       float64            
 6   volume                         Int64              
 7   ticker                         object             
 8   id                             object             
 9   SMA_20                         float64            
 10  EMA_50                         float64            
 11  WILLR                          float64            
 12  ATR                            float64            
 13  ADX                            float64    

In [21]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df_final, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
